In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

%matplotlib inline

df = pd.read_csv('subreddit_data.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,8,19,27,79,80,81,82,83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.shape

(19500, 85)

In [4]:
df = df[['subreddit', 'selftext', 'title']]

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# drop all missing values
df.dropna()

,subreddit,selftext,title
0,cars,[removed],Looking for an inexpensive bluetooth stereo co...
1,cars,I personally Think it’s Brooks From Dragtimes ...,Who produces the most underrated Car content o...
3,cars,I'm considering doing it and would love to hea...,Anyone have any direct experience selling a ca...
4,cars,"I apologize for the rant, but I seriously need...",Can any former or current car dealership mecha...
6,cars,[removed],Automotive News: Toyota axing Avalon sedan aft...
...,...,...,...
19489,travel,[removed],Anti-Asian racism
19490,travel,[removed],Book The Best Travel journey in UK from Luftha...
19493,travel,[removed],Global Entry Question
19495,travel,"Hey guys, in July me and a couple of friends p...",First Time Las Vegas Trip With My Boys.


In [7]:
df = (df[~df['subreddit'].isna()])
df = (df[~df['selftext'].isna()])
df = (df[~df['title'].isna()])

In [8]:
# remove 'removed', 'deleted'
df = df[df['selftext']!= '[removed]']
df = df[df['selftext']!='deleted']
df = df[~df['selftext'].str.startswith('[removed]')]

In [9]:
from bs4 import BeautifulSoup
import re
import string

In [10]:
# remove URL by using regex from colums selftext, title
df['selftext'] = df['selftext'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', regex=True)
df['title'] = df['title'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', regex=True)

In [11]:
# Remove certain reddit markdown issues from title and selftext column
for column in ['title','selftext','subreddit']:
    df[column] = df[column].str.replace('\xa0',' ', regex=True).replace('&lt;',' ', regex=True).replace('&gt;',' ', regex=True).replace('&amp;',' ', regex=True).replace('x200B','', regex=True).replace('\n',' ', regex=True)

In [12]:
# Remove punctuation manually:
string.punctuation += '’'
df['selftext'] = ["".join(l for l in s if l not in string.punctuation) for s in df['selftext']]
df['title'] = ["".join(l for l in s if l not in string.punctuation) for s in df['title']]
df['subreddit'] = [''.join(l for l in s if l not in string.punctuation) for s in df['subreddit']]

In [13]:
df['subreddit'].value_counts()

cars      2585
travel    2372
Name: subreddit, dtype: int64

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
# remove multiple consective blanks or spaces 
import re
df['selftext'] = [re.sub(' +', ' ', text) for text in df['selftext']]
df['title'] = [re.sub(' +', ' ', text) for text in df['title']]

In [16]:
df.head()

,subreddit,selftext,title
1,cars,I personally Think its Brooks From Dragtimes o...,Who produces the most underrated Car content o...
3,cars,Im considering doing it and would love to hear...,Anyone have any direct experience selling a ca...
4,cars,I apologize for the rant but I seriously need ...,Can any former or current car dealership mecha...
11,cars,Surely whichever car manufacturer makes one co...,Isnt there a market for a small efficient elec...
15,cars,What would be a good choice for under 17000 th...,What do you think about Toyota


In [17]:
df.shape

(4957, 3)

In [18]:
# drop rows with df['selftext'] == deleted
df = df.loc[df['selftext'] != 'deleted']

In [19]:
# just like the nlp lab, we check the lenght and word counts
#df['selftext_length'] = df['selftext'].map(lambda x: len(x))

In [20]:
df['selftext_word_count'] = df['selftext'].map(lambda x: len(x.split()))

In [21]:
# check df['title'] has float values , so change to string to do the length count
#df['title'] = df['title'].astype(str)

In [22]:
#df['title_length'] = df['title'].map(lambda x: len(x))

In [23]:
#df['title_word_count'] = df['selftext'].map(lambda x: len(x.split()))

In [24]:
# remove selftext columns with space
#df = df.loc[df['selftext'] != '   ']

In [25]:
#df['selftext'] = df['selftext'].replace('\s+', ' ', regex=True)

In [26]:
df.shape

(4863, 4)

In [27]:
df.drop([16249, 17566, 11369, 17614],  inplace = True)

In [28]:
df.sort_values(by='selftext_word_count', ascending= False)['selftext'].head(10)

9925     While vaccines are starting to be administered...
16031    While vaccines are starting to be administered...
13034    While vaccines are starting to be administered...
19029    While vaccines are starting to be administered...
17120    For the last few months weve been getting mult...
16400    Sometimes you watch a movie and it hits you so...
17217    Hi guys so Im planning a two persons 1 month t...
11189     I recently came back from a 5 day trip to vis...
11567    Hello I have a problem involving money school ...
9598     I want to preface this by saying that this pos...
Name: selftext, dtype: object

In [29]:
df.drop(columns = 'selftext_word_count', axis=1)

,subreddit,selftext,title
1,cars,I personally Think its Brooks From Dragtimes o...,Who produces the most underrated Car content o...
3,cars,Im considering doing it and would love to hear...,Anyone have any direct experience selling a ca...
4,cars,I apologize for the rant but I seriously need ...,Can any former or current car dealership mecha...
11,cars,Surely whichever car manufacturer makes one co...,Isnt there a market for a small efficient elec...
15,cars,What would be a good choice for under 17000 th...,What do you think about Toyota
...,...,...,...
19469,travel,Im hoping to travel to Italy at some point in...,Thoughts on these Italian Cities
19470,travel,Iceland is now open to fully vaccinated people...,Does anybody have any good recommendations for...
19478,travel,The title is very vague but I will try to expl...,What are the odds of being able to travel from...
19495,travel,Hey guys in July me and a couple of friends pl...,First Time Las Vegas Trip With My Boys


In [30]:
df.to_csv('subreddit_clean.csv')

In [31]:
# remove non-ASCII characters from title and selftext columns
df['title'] = df['title'].str.encode('ascii', errors='ignore').str.decode('ascii')

In [32]:
df['selftext'] = df['selftext'].str.encode('ascii', errors='ignore').str.decode('ascii')

In [33]:
df.head()

,subreddit,selftext,title,selftext_word_count
1,cars,I personally Think its Brooks From Dragtimes o...,Who produces the most underrated Car content o...,24
3,cars,Im considering doing it and would love to hear...,Anyone have any direct experience selling a ca...,29
4,cars,I apologize for the rant but I seriously need ...,Can any former or current car dealership mecha...,373
11,cars,Surely whichever car manufacturer makes one co...,Isnt there a market for a small efficient elec...,67
15,cars,What would be a good choice for under 17000 th...,What do you think about Toyota,28


In [ ]:
# remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['title'] = df['title'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))
df['selftext'] = df['selftext'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))

In [ ]:
import pandas as pd
df.to_csv('clean_final_data.csv')